In [21]:
#import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
#import niftynet as nn
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk

#original_path = '/proj/NIRAL/users/siyangj/InfantTissuePreTraining/NeoSeg/'
#T0088-1-1-neo_labels_seg.nii.gz
#T0088-1-1-neo-T1-RAI-regAtlas-crop_stripped.nii.gz
#T0088-1-1-neo-T2-regT1Atlas_stripped.nii.gz
original_path = '/proj/NIRAL/users/siyangj/InfantTissuePreTraining/MultiAtlas/'
target_path = '/proj/NIRAL/users/siyangj/myPretrainData/'

In [3]:
min_dim = [96,112,96]

In [23]:
print(original_path)

/proj/NIRAL/users/siyangj/InfantTissuePreTraining/MultiAtlas/


In [24]:
for filename in os.listdir(original_path):
    if filename.find('Labels')==-1 and filename.find('labels')==-1:
        continue
    if filename[:3]=='neo':
        try:
            ID = filename[:10]
            im_T1 = sitk.ReadImage(original_path+ID+'-T1-RAI-regAtlas-crop_stripped.nii.gz')
            im_T2 = sitk.ReadImage(original_path+ID+'-T2-regT1Atlas_stripped.nii.gz')
            im_seg = sitk.ReadImage(original_path+ID+'_labels_seg.nii.gz')
        except RuntimeError:
            print(filename)
            continue
    elif filename[0]=='T':
        try:
            ID = filename[:9]
            im_T1 = sitk.ReadImage(original_path+ID+'-neo-T1-RAI-regAtlas-crop_stripped.nii.gz')
            im_T2 = sitk.ReadImage(original_path+ID+'-neo-T2-regT1Atlas_stripped.nii.gz')
            im_seg = sitk.ReadImage(original_path+ID+'-neo_labels_seg.nii.gz')
        except RuntimeError:
            print(filename)
            continue
    elif filename[:4]=='BUSS':
        try:
            ID = filename[:9]
            im_T1 = sitk.ReadImage(original_path+ID+'_T1_reg-stripped_scaled.nii.gz')
            im_T2 = sitk.ReadImage(original_path+ID+'_T2_reg-stripped_scaled.nii.gz')
            im_seg = sitk.ReadImage(original_path+ID+'_neo-seg-3Labels.nii.gz')
        except RuntimeError:
            print(filename)
            continue
    else:
        print(filename)
        continue
    
    assert(im_T1.GetSize()==im_T2.GetSize() and im_T1.GetSize()==im_seg.GetSize())
    im_size = im_T1.GetSize()
    
    dim_diff = [0,0,0]
    pad_l = [0,0,0]
    pad_u = [0,0,0]
        
    for d in range(3):
        if (im_size[d] < min_dim[d]):
            dim_diff[d] = min_dim[d]-im_size[d]
            pad_l[d] = round(dim_diff[d]/2)
            pad_u[d] = dim_diff[d]-pad_l[d]
            
    #print((i,pad_l,pad_u))
    im_T1 = sitk.ConstantPad(im_T1,pad_l,pad_u,constant=0)
    im_T2 = sitk.ConstantPad(im_T2,pad_l,pad_u,constant=0)
    im_seg = sitk.ConstantPad(im_seg,pad_l,pad_u,constant=0)
    
    dim_diff = [0,0,0]
    pad_l = [0,0,0]
    pad_u = [0,0,0]
    for d in range(3):
        if (im_size[d] > min_dim[d]):
            dim_diff[d] = im_size[d]-min_dim[d]
            pad_l[d] = round(dim_diff[d]/2)
            pad_u[d] = dim_diff[d]-pad_l[d]
    
    #print((i,pad_l,pad_u))
    im_new_T1 = sitk.Crop(im_T1,pad_l,pad_u)
    sitk.WriteImage(im_new_T1,target_path+'T1/'+ID+'.nrrd')
    im_new_T2 = sitk.Crop(im_T2,pad_l,pad_u)
    sitk.WriteImage(im_new_T2,target_path+'T2/'+ID+'.nrrd')
    im_new_seg = sitk.Crop(im_seg,pad_l,pad_u)
    sitk.WriteImage(im_new_seg,target_path+'seg/'+ID+'.nrrd')
    
    

In [39]:
for filename in os.listdir(target_path+'seg/'): 
    arr = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(target_path,'seg',filename)))
    labels,counts=np.unique(arr,return_counts=True)
    print(np.round(1/counts * 1e6))
    for l,c in zip(labels,-np.round(1/counts * 1e6)):
        arr[arr==l] = c
    arr = np.round(-arr)
    print(np.unique(arr))

    sitk.WriteImage(sitk.GetImageFromArray(arr),target_path+'weight/'+filename[:-5]+'.nrrd')

[ 2.  8.  5. 24.]
[ 2  5  8 24]
[ 2.  6.  4. 13.]
[ 2  4  6 13]
[ 2.  6.  5. 17.]
[ 2  5  6 17]
[ 2.  6.  4. 18.]
[ 2  4  6 18]
[ 2.  7.  5. 22.]
[ 2  5  7 22]
[ 2.  6.  5. 23.]
[ 2  5  6 23]
[ 2.  6.  3. 14.]
[ 2  3  6 14]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  7.  5. 26.]
[ 2  5  7 26]
[ 1.  8.  6. 24.]
[ 1  6  8 24]
[ 2.  6.  4. 17.]
[ 2  4  6 17]
[ 2.  7.  5. 19.]
[ 2  5  7 19]
[ 2.  6.  5. 23.]
[ 2  5  6 23]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  6.  4. 20.]
[ 2  4  6 20]
[ 1.  8.  5. 26.]
[ 1  5  8 26]
[ 2.  7.  3. 12.]
[ 2  3  7 12]
[ 2.  6.  5. 25.]
[ 2  5  6 25]
[ 2.  7.  5. 20.]
[ 2  5  7 20]
[ 2.  7.  5. 23.]
[ 2  5  7 23]
[ 2.  6.  5. 30.]
[ 2  5  6 30]
[ 1.  8.  6. 25.]
[ 1  6  8 25]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  5.  4. 21.]
[ 2  4  5 21]
[ 1.  7.  6. 29.]
[ 1  6  7 29]
[ 2.  6.  5. 18.]
[ 2  5  6 18]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  8.  5. 19.]
[ 2  5  8 19]
[ 2.  7.  4. 22.]
[ 2  4  7 22]
[ 2.  6.  5. 32.]
[ 2  5  6 32]
[ 1.  7.  6. 24.]
[ 1  6  7 24]
[ 1.  8.

[ 2.  7.  4. 18.]
[ 2  4  7 18]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  6.  5. 26.]
[ 2  5  6 26]
[ 2.  6.  3. 20.]
[ 2  3  6 20]
[ 2.  7.  5. 32.]
[ 2  5  7 32]
[ 2.  7.  5. 22.]
[ 2  5  7 22]
[ 1.  8.  6. 35.]
[ 1  6  8 35]
[ 2.  7.  5. 31.]
[ 2  5  7 31]
[ 2.  8.  5. 24.]
[ 2  5  8 24]
[ 1.  9.  7. 27.]
[ 1  7  9 27]
[ 2.  7.  4. 19.]
[ 2  4  7 19]
[ 2.  7.  5. 25.]
[ 2  5  7 25]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  6.  4. 33.]
[ 2  4  6 33]
[ 2.  6.  4. 21.]
[ 2  4  6 21]
[ 2.  7.  4. 26.]
[ 2  4  7 26]
[ 1.  7.  5. 29.]
[ 1  5  7 29]
[ 2.  6.  4. 23.]
[ 2  4  6 23]
[ 2.  6.  4. 24.]
[ 2  4  6 24]
[ 2.  7.  5. 35.]
[ 2  5  7 35]
[ 2.  6.  4. 20.]
[ 2  4  6 20]
[ 2.  7.  5. 26.]
[ 2  5  7 26]
[ 1.  7.  5. 27.]
[ 1  5  7 27]
[ 2.  8.  5. 27.]
[ 2  5  8 27]
[ 2.  7.  5. 25.]
[ 2  5  7 25]
[ 2.  6.  4. 23.]
[ 2  4  6 23]
[ 2.  5.  4. 14.]
[ 2  4  5 14]
[ 2.  7.  5. 20.]
[ 2  5  7 20]
[ 2.  7.  5. 19.]
[ 2  5  7 19]
[ 2.  7.  5. 30.]
[ 2  5  7 30]
[ 2.  6.  4. 21.]
[ 2  4  6 21]
[ 1.  7.

[ 2.  7.  5. 30.]
[ 2  5  7 30]
[ 2.  7.  5. 29.]
[ 2  5  7 29]
[ 2.  7.  5. 18.]
[ 2  5  7 18]
[ 2.  6.  4. 13.]
[ 2  4  6 13]
[ 1.  8.  6. 28.]
[ 1  6  8 28]
[ 1.  9.  6. 32.]
[ 1  6  9 32]
[ 2.  6.  5. 26.]
[ 2  5  6 26]
[ 2.  6.  4. 21.]
[ 2  4  6 21]
[ 1.  8.  6. 41.]
[ 1  6  8 41]
[ 2.  7.  5. 28.]
[ 2  5  7 28]
[ 1.  8.  6. 26.]
[ 1  6  8 26]
[ 2.  5.  4. 22.]
[ 2  4  5 22]
[ 1.  7.  5. 25.]
[ 1  5  7 25]
[ 1.  9.  7. 28.]
[ 1  7  9 28]
[ 1.  9.  6. 32.]
[ 1  6  9 32]
[ 2.  5.  3. 17.]
[ 2  3  5 17]
[ 2.  6.  3. 14.]
[ 2  3  6 14]
[ 1.  9.  6. 33.]
[ 1  6  9 33]
[ 2.  6.  5. 16.]
[ 2  5  6 16]
[ 2.  6.  5. 32.]
[ 2  5  6 32]
[ 1.  8.  6. 31.]
[ 1  6  8 31]
[ 2.  6.  5. 20.]
[ 2  5  6 20]
[ 2.  7.  5. 32.]
[ 2  5  7 32]
[ 1.  7.  5. 28.]
[ 1  5  7 28]
[ 2.  6.  6. 26.]
[ 2  6 26]
[ 2.  7.  5. 16.]
[ 2  5  7 16]
[ 1.  9.  7. 27.]
[ 1  7  9 27]
[ 1.  7.  5. 38.]
[ 1  5  7 38]
[ 2.  5.  4. 16.]
[ 2  4  5 16]
[ 1.  7.  5. 37.]
[ 1  5  7 37]
[ 2.  6.  4. 23.]
[ 2  4  6 23]
[ 2.  6.  5

[ 2.  7.  4. 15.]
[ 2  4  7 15]
[ 2.  7.  5. 30.]
[ 2  5  7 30]
[ 2.  7.  5. 21.]
[ 2  5  7 21]
[ 2.  6.  4. 18.]
[ 2  4  6 18]
[ 1.  8.  6. 36.]
[ 1  6  8 36]
[ 2.  7.  5. 25.]
[ 2  5  7 25]
[ 2.  6.  4. 13.]
[ 2  4  6 13]
[ 1.  7.  6. 30.]
[ 1  6  7 30]
[ 2.  7.  5. 29.]
[ 2  5  7 29]
[ 2.  7.  5. 22.]
[ 2  5  7 22]
[ 2.  6.  5. 18.]
[ 2  5  6 18]
[ 1.  8.  6. 27.]
[ 1  6  8 27]
[ 2.  7.  5. 21.]
[ 2  5  7 21]
[ 1.  7.  5. 26.]
[ 1  5  7 26]
[ 2.  6.  4. 23.]
[ 2  4  6 23]
[ 1.  8.  5. 28.]
[ 1  5  8 28]
[ 2.  5.  5. 19.]
[ 2  5 19]
[ 2.  6.  5. 24.]
[ 2  5  6 24]
[ 2.  5.  4. 22.]
[ 2  4  5 22]
[ 2.  6.  4. 16.]
[ 2  4  6 16]
[ 2.  6.  5. 26.]
[ 2  5  6 26]
[ 2.  6.  5. 22.]
[ 2  5  6 22]
[ 2.  7.  4. 24.]
[ 2  4  7 24]
[ 2.  8.  5. 22.]
[ 2  5  8 22]
[ 2.  6.  5. 30.]
[ 2  5  6 30]
[ 2.  8.  4. 20.]
[ 2  4  8 20]
[ 1.  8.  6. 24.]
[ 1  6  8 24]
[ 1.  8.  5. 26.]
[ 1  5  8 26]
[ 2.  6.  6. 22.]
[ 2  6 22]
[ 2.  6.  5. 28.]
[ 2  5  6 28]
[ 1.  7.  6. 34.]
[ 1  6  7 34]
[ 2.  8.  4. 2

In [38]:
np.round(2.500001)

3.0